# Yellow Cab Pick-Up Points Joined with Overture Maps Buildings
## We take hexagons from an H3 heatmap of TLC Data, processing millions of records in seconds
## We then obtain the building geometries from Overture using DuckDB and finally join the two datasets to plot with Lonboard

Install dependencies.

In [1]:
! pip install lonboard geopandas matplotlib palettable numpy duckdb 

In [1]:
import shapely 
import geopandas as gpd
import duckdb
import numpy as np
from matplotlib.colors import LogNorm
from palettable.matplotlib import Inferno_9
from lonboard import Map, PolygonLayer
from lonboard.colormap import apply_continuous_cmap

This is a helpful snippet I reuse almost everyay.

In [2]:
con = duckdb.connect()
con.sql(""" INSTALL h3 FROM community;
            LOAD h3;
            INSTALL spatial;
            LOAD spatial;
            INSTALL httpfs;
            LOAD httpfs;
            SET s3_region='us-west-2';""")

I go more into detail in [this notebook](https://github.com/kentstephen/duckdb_h3/blob/main/tlc.ipynb). I borrow inspiration and some code from several NYC Fused UDFs.

In [3]:
url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-01.parquet'
min_cnt = 20
resolution = 10

query = f"""
WITH to_cells as (
SELECT
    h3_h3_to_string(h3_latlng_to_cell(pickup_latitude, pickup_longitude, {resolution})) AS cell_id, 
    h3_cell_to_boundary_wkt(cell_id) boundary,
    count(1) as cnt
FROM read_parquet('{url}')
GROUP BY 1 
HAVING cnt >= {min_cnt})
SELECT
    boundary, -- Kepler in Jupyter doesn't like the H3 string so we have to use the Polygon boundary
    cnt
FROM to_cells

"""

df = con.sql(query).df()
gdf_cab = gpd.GeoDataFrame(df.drop(columns=['boundary']), geometry=df.boundary.apply(shapely.wkt.loads))


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
gdf_cab = gdf_cab.set_crs(epsg=4326, inplace=True)
print(gdf_cab)

        cnt                                           geometry
0     13004  POLYGON ((-73.95648 40.76834, -73.95739 40.768...
1      6547  POLYGON ((-74.00543 40.74549, -74.00634 40.745...
2       492  POLYGON ((-74.01573 40.7048, -74.01664 40.7046...
3     12196  POLYGON ((-73.99001 40.74213, -73.99092 40.741...
4      9120  POLYGON ((-73.9625 40.77384, -73.96341 40.7736...
...     ...                                                ...
6329     29  POLYGON ((-73.96098 40.75046, -73.9619 40.7503...
6330     21  POLYGON ((-73.96346 40.79082, -73.96438 40.790...
6331     20  POLYGON ((-73.93779 40.70351, -73.9387 40.7033...
6332     39  POLYGON ((-73.92155 40.73452, -73.92246 40.734...
6333     21  POLYGON ((-73.94682 40.75575, -73.94773 40.755...

[6334 rows x 2 columns]


You can also find a bbox by querying Overture's land division type, but here we want some outliers out of the city so we are going to use this bbox generated with [this bounding box tool](https://boundingbox.klokantech.com/). Select the CSV option.

In [5]:
bbox_string = "-74.27507,40.488386,-73.72905,40.957151" #NYC

# Split the string into individual float values
bbox_values = list(map(float, bbox_string.split(',')))

# Create a dictionary with the required keys and values
bbox = {
    'xmin': bbox_values[0],
    'xmax': bbox_values[2],
    'ymin': bbox_values[1],
    'ymax': bbox_values[3]
}

print(bbox)

{'xmin': -74.27507, 'xmax': -73.72905, 'ymin': 40.488386, 'ymax': 40.957151}


This is a pretty basic Overture DuckDB query. If you want to examine more columns you can check out the [buildings schema here](https://docs.overturemaps.org/schema/reference/buildings/building/).

In [6]:
query = f"""
SELECT
    ST_AsText(geometry) as geometry
FROM read_parquet('s3://overturemaps-us-west-2/release/2024-09-18.0/theme=buildings/type=building/*', filename=true, hive_partitioning=1)
WHERE
    bbox.xmin <= {bbox["xmax"]}
    AND bbox.xmax >= {bbox["xmin"]}
    AND bbox.ymin <= {bbox["ymax"]}
    AND bbox.ymax >= {bbox["ymin"]}
  
"""
df_buildings = con.sql(query).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Geometries moving between Pandas and DuckDB need to be as text, so here we apply `shapely.wkt.loads` so we can make a GeoPandas GeoDataFrame.

In [7]:
df_buildings['geometry'] = df_buildings['geometry'].apply(shapely.wkt.loads)

Make a GeoDataFrame and set the CRS.

In [8]:
gdf_buildings = gpd.GeoDataFrame(df_buildings, geometry='geometry')
gdf_buildings.set_crs(epsg=4326, inplace=True)

,geometry
0,"POLYGON ((-74.27378 40.4893, -74.27395 40.4892..."
1,"POLYGON ((-74.27436 40.501, -74.27434 40.50108..."
2,"POLYGON ((-74.27357 40.50101, -74.27349 40.500..."
3,"POLYGON ((-74.27318 40.5009, -74.27309 40.5008..."
4,"POLYGON ((-74.27318 40.50097, -74.27307 40.500..."
...,...
1746236,"POLYGON ((-73.73109 40.95716, -73.73099 40.957..."
1746237,"POLYGON ((-73.73077 40.95716, -73.73084 40.957..."
1746238,"POLYGON ((-73.73049 40.95711, -73.73053 40.957..."
1746239,"POLYGON ((-73.73011 40.95716, -73.73015 40.957..."


Here we use spatial join to merge the cabs with the buildings and drop an unneccesary index.

In [9]:
gdf_joined = gdf_buildings.sjoin(gdf_cab)
gdf_joined = gdf_joined.set_crs(epsg=4326)
gdf_joined = gdf_joined.drop(columns=['index_right'])
print(gdf_joined)

                                                  geometry  cnt
36822    POLYGON ((-74.22391 40.57405, -74.22399 40.574...   29
36825    POLYGON ((-74.22399 40.57412, -74.22392 40.574...   29
36826    POLYGON ((-74.22388 40.57415, -74.22382 40.574...   29
36827    POLYGON ((-74.22437 40.57442, -74.22421 40.574...   29
36828    POLYGON ((-74.22409 40.57438, -74.22403 40.574...   29
...                                                    ...  ...
1687812  POLYGON ((-73.81267 40.78368, -73.81266 40.783...  503
1687813  POLYGON ((-73.81295 40.78383, -73.81294 40.783...  503
1687814  POLYGON ((-73.81287 40.78382, -73.81285 40.783...  503
1687815  POLYGON ((-73.81275 40.78379, -73.81273 40.783...  503
1688063  POLYGON ((-73.81404 40.78397, -73.81403 40.784...  503

[134270 rows x 2 columns]


I got the following code [from this Lonboard Overture Maps Example](https://developmentseed.org/lonboard/latest/examples/overture-maps/). Kyle Barron has great docs.

In [10]:
counts = gdf_joined["cnt"].to_numpy()
counts = np.nan_to_num(counts, nan=1)
normalizer = LogNorm(1, counts.max(), clip=True)
normalized_counts = normalizer(counts)

In [11]:
Inferno_9.mpl_colormap
colors = apply_continuous_cmap(normalized_counts, Inferno_9)

In [12]:
layer = PolygonLayer.from_geopandas(
    gdf_joined,
    get_fill_color=colors,  
)
m = Map(layer)

Render the map. You can drag to zoom in.

In [13]:
m

Map(layers=[PolygonLayer(get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x00000216C0EFBDC0>
[
  [
  …

Add a dark basemap

In [15]:
m.basemap_style="https://basemaps.cartocdn.com/gl/dark-matter-nolabels-gl-style/style.json"

### I think this is a cool template to join H3 with GeoJSON. What big dataset do you have in mind for this?